<a href="https://colab.research.google.com/github/nilu-ai/Restaurant_Fttx_project/blob/main/Pre%20trained%20Model%20wala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 11.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=16b140ea6e9e47bbbba9cb4d0d6178b71dc65a72bcad25011959287855b34682
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [2]:
!unzip trainss

Archive:  trainss.zip
   creating: trainss/abhi/
  inflating: trainss/abhi/WIN_20240524_15_28_37_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_39_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_41_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_42_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_44_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_45_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_47_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_48_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_50_Pro.jpg  
  inflating: trainss/abhi/WIN_20240524_15_28_52_Pro.jpg  
   creating: trainss/Abhishek 1/
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_24_Pro (2).jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_24_Pro.jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_25_Pro (2).jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_28_25_Pro.jpg  
  inflating: trainss/Abhishek 1/WIN_20240524_15_29_22_Pro.jpg  
  inflatin

In [19]:
from deepface import DeepFace
df = DeepFace.find(img_path = "/content/WIN_20240531_13_34_12_Pro.jpg", db_path = "/content/trainss",model_name="Facenet512")
print(df)
if df[0].distance.empty:
  print("Unkonw")
elif df[0].distance[0]<0.3:
  print(df)
else:
  print("Unkonw")



24-05-31 08:07:10 - Searching /content/WIN_20240531_13_34_12_Pro.jpg in 34 length datastore
24-05-31 08:07:11 - find function duration 0.7983593940734863 seconds
[Empty DataFrame
Columns: [identity, hash, target_x, target_y, target_w, target_h, source_x, source_y, source_w, source_h, threshold, distance]
Index: []]
Unkonw


In [11]:
from flask import Flask, request, jsonify
import cv2
import numpy as np
import base64
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from google.colab.patches import cv2_imshow
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))
app = Flask(__name__)


# Dictionary which assigns each label an emotion



def predict_emotion(face_img):
    predict=DeepFace.find(img_path = face_img, db_path = "/content/Train",model_name="Facenet512")
    print(predict)

@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        data = request.json
        img_data = data['image'].split(',')[1]
        nparr = np.frombuffer(base64.b64decode(img_data), np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        # face_images, coordinates = detect_and_extract_faces(img, face_cascade)

        # predictions = []
        # for face_img, (x, y, w, h) in zip(face_images, coordinates):
        #     cv2.flip(face_img,0)
        #     cv2_imshow(face_img)

        #     label, confidence = predict_emotion(face_img)
        #     print(predict_emotion(face_img))
        #     predictions.append({
        #         'label': label,
        #         'confidence': confidence,

        #     })
        df="unknow"
        predict=DeepFace.find(img_path = img, db_path = "/content/trainss",model_name="Facenet512")
        if predict[0].distance.empty:
          print("Unkonw")
        elif predict[0].distance[0]<0.3:
          print(df)
          df=predict
        else:
          print("Unkonw")

        return jsonify(df)
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <video id="video" width="640" height="480" autoplay></video>
        <canvas id="canvas" width="640" height="480" style="display:none;"></canvas>
        <div id="prediction"></div>
        <script>
            const video = document.getElementById('video');
            const canvas = document.getElementById('canvas');
            const predictionDiv = document.getElementById('prediction');

            async function initCamera() {
                try {
                    const stream = await navigator.mediaDevices.getUserMedia({ video: true });
                    video.srcObject = stream;
                } catch (err) {
                    console.error('Error accessing camera:', err);
                }
            }

            async function captureAndPredict() {
                const context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                const imageData = canvas.toDataURL('image/jpeg');
                const response = await fetch('/', {
                    method: 'POST',
                    body: JSON.stringify({ image: imageData }),
                    headers: {
                        'Content-Type': 'application/json'
                    }
                });
                const result = await response.json();

                predictionDiv.innerHTML = JSON.stringify(result, null, 2);
            }

            setInterval(captureAndPredict, 1000);

            initCamera();
        </script>
    </body>
    </html>
    """

if __name__ == '__main__':
    app.run(host='0.0.0.0')


https://svbiubpeh6b-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


24-05-31 08:03:35 - Found 1 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]
ERROR:__main__:Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "<ipython-input-11-0acd1495442d>", line 44, in upload_image
    predict=DeepFace.find(img_path = img, db_path = "/content/trainss",model_name="Facenet512")
  File "/usr/local/lib/python3.10/dist-packages/deepface/DeepFace.py", line 320, in find
    return recognition.find(
  Fil

24-05-31 08:03:36 - There are now 34 representations in ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl
24-05-31 08:03:36 - Searching [[[156 158 152]
  [156 158 152]
  [156 158 152]
  ...
  [231 230 220]
  [231 231 219]
  [231 231 219]]

 [[155 157 151]
  [155 157 151]
  [156 158 152]
  ...
  [230 229 219]
  [230 230 218]
  [230 230 218]]

 [[154 155 151]
  [155 156 152]
  [153 157 152]
  ...
  [229 228 218]
  [229 228 218]
  [229 228 218]]

 ...

 [[160 157 142]
  [159 156 141]
  [158 155 140]
  ...
  [167 179 183]
  [165 177 183]
  [163 175 181]]

 [[160 158 140]
  [159 157 139]
  [158 155 140]
  ...
  [169 181 185]
  [168 178 185]
  [166 176 183]]

 [[163 161 143]
  [161 159 141]
  [158 155 140]
  ...
  [169 181 185]
  [168 178 185]
  [167 177 184]]] in 34 length datastore


...

24-05-31 08:03:49 - Searching [[[185 185 173]
  [185 185 173]
  [184 183 173]
  ...
  [203 203 203]
  [203 203 203]
  [202 202 202]]

 [[185 185 173]
  [185 185 173]
  [184 183 173]
  ...
  [202 202 202]
  [201 201 201]
  [200 200 200]]

 [[185 185 173]
  [185 185 173]
  [183 184 174]
  ...
  [201 202 200]
  [199 200 198]
  [197 198 196]]

 ...

 [[208 209 199]
  [208 209 199]
  [209 210 200]
  ...
  [181 199 200]
  [186 201 204]
  [182 197 200]]

 [[211 209 201]
  [211 209 201]
  [211 209 201]
  ...
  [181 198 201]
  [183 200 203]
  [179 196 199]]

 [[211 209 201]
  [211 209 201]
  [211 209 201]
  ...
  [183 200 203]
  [183 200 203]
  [179 196 199]]] in 34 length datastore
24-05-31 08:03:50 - Searching [[[181 185 174]
  [181 185 174]
  [179 185 174]
  ...
  [222 225 216]
  [222 224 218]
  [222 224 218]]

 [[181 185 174]
  [181 185 174]
  [179 185 174]
  ...
  [221 224 215]
  [222 224 218]
  [222 224 218]]

 [[181 185 174]
  [181 185 174]
  [179 184 175]
  ...
  [223 224 215]
  [223 22

INFO:werkzeug:127.0.0.1 - - [31/May/2024 08:03:50] "POST / HTTP/1.1" 200 -


24-05-31 08:03:50 - find function duration 0.3465123176574707 seconds
Unkonw
24-05-31 08:03:51 - Searching [[[173 184 176]
  [173 184 176]
  [173 184 176]
  ...
  [222 221 211]
  [222 221 211]
  [221 220 210]]

 [[173 184 176]
  [173 184 176]
  [173 184 176]
  ...
  [220 219 209]
  [221 220 210]
  [222 221 211]]

 [[176 184 177]
  [176 184 177]
  [176 184 177]
  ...
  [219 217 209]
  [220 218 210]
  [221 219 211]]

 ...

 [[224 213 186]
  [222 213 186]
  [219 209 185]
  ...
  [217 234 231]
  [215 230 232]
  [214 229 231]]

 [[225 214 186]
  [222 213 186]
  [218 208 184]
  ...
  [219 233 231]
  [215 230 232]
  [212 227 229]]

 [[223 212 184]
  [220 212 183]
  [218 208 184]
  ...
  [221 235 233]
  [216 231 233]
  [213 228 230]]] in 34 length datastore


INFO:werkzeug:127.0.0.1 - - [31/May/2024 08:03:51] "POST / HTTP/1.1" 200 -


24-05-31 08:03:51 - find function duration 0.428363561630249 seconds
Unkonw
24-05-31 08:03:52 - Searching [[[177 187 175]
  [176 186 174]
  [176 186 174]
  ...
  [219 219 213]
  [219 218 214]
  [218 217 213]]

 [[176 186 174]
  [176 186 174]
  [176 186 174]
  ...
  [219 219 213]
  [218 217 213]
  [218 217 213]]

 [[175 184 174]
  [175 184 174]
  [176 186 174]
  ...
  [218 218 212]
  [218 217 213]
  [218 217 213]]

 ...

 [[218 211 186]
  [220 213 186]
  [217 212 181]
  ...
  [223 235 229]
  [219 233 229]
  [219 234 230]]

 [[217 209 186]
  [220 213 188]
  [217 212 181]
  ...
  [223 235 229]
  [219 233 229]
  [219 233 229]]

 [[219 211 188]
  [222 215 190]
  [216 211 180]
  ...
  [224 237 229]
  [221 235 231]
  [219 233 229]]] in 34 length datastore


In [11]:
df.verify(img_path ="/content/WIN_20240524_17_49_35_Pro.jpg")

AttributeError: 'list' object has no attribute 'verify'

In [6]:
new_df=DeepFace.stream(db_path = "/content/Train")


24-05-31 05:40:09 - age_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/age_model_weights.h5
To: /root/.deepface/weights/age_model_weights.h5
100%|██████████| 539M/539M [00:02<00:00, 207MB/s]


24-05-31 05:40:14 - Age model is just built
24-05-31 05:40:16 - gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5
100%|██████████| 537M/537M [00:08<00:00, 63.7MB/s]


24-05-31 05:40:27 - Gender model is just built
24-05-31 05:40:27 - facial_expression_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facial_expression_model_weights.h5
To: /root/.deepface/weights/facial_expression_model_weights.h5
100%|██████████| 5.98M/5.98M [00:00<00:00, 21.0MB/s]


24-05-31 05:40:28 - Emotion model is just built
24-05-31 05:40:28 - VGG-Face is built


In [29]:

import pickle
model = pickle.load(open('/content/Train/ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl','rb'))

AttributeError: 'list' object has no attribute 'predict'

In [21]:
import pickle
import numpy as np
from PIL import Image
import cv2

# Step 1: Load the saved model
with open('/content/Train/ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl', 'rb') as file:
    model = pickle.load(file)

# Step 2: Preprocess the new image
def preprocess_image(image_path):
    # Load image
    image = Image.open(image_path)
    image = image.convert('RGB')
    # Resize image to the size expected by the FaceNet model (usually 160x160)
    image = image.resize((160, 160))
    # Convert image to numpy array
    image = np.array(image)
    # Normalize the image (as per the normalization done during training)
    image = image.astype('float32')
    mean, std = image.mean(), image.std()
    image = (image - mean) / std
    # Expand dimensions to match the input shape of the model (batch size, height, width, channels)
    image = np.expand_dims(image, axis=0)
    return image

# Path to the new image
new_image_path = '/content/WIN_20240514_09_48_51_Pro.jpg'
# Preprocess the new image
processed_image = preprocess_image(new_image_path)
pre=model.predict(processed_image)
print(pre)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Train/ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl'

In [22]:
from flask import Flask, request, jsonify
import cv2
import numpy as np
import base64
from deepface import DeepFace  # Ensure you have deepface installed

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        data = request.json
        img_data = data['image'].split(',')[1]
        nparr = np.frombuffer(base64.b64decode(img_data), np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        try:
            # Perform face recognition using DeepFace
            predictions = DeepFace.find(img_path=img, db_path="/content/trainss", model_name="Facenet512")

            if not predictions[0].empty and predictions[0].distance[0] < 0.3:
                response = {
                    'status': 'success',
                    'data': predictions[0].to_dict()
                }
            else:
                response = {
                    'status': 'unknown'
                }
        except Exception as e:
            response = {
                'status': 'error',
                'message': str(e)
            }

        return jsonify(response)

    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <video id="video" width="640" height="480" autoplay></video>
        <canvas id="canvas" width="640" height="480" style="display:none;"></canvas>
        <div id="prediction"></div>
        <script>
            const video = document.getElementById('video');
            const canvas = document.getElementById('canvas');
            const predictionDiv = document.getElementById('prediction');

            async function initCamera() {
                try {
                    const stream = await navigator.mediaDevices.getUserMedia({ video: true });
                    video.srcObject = stream;
                } catch (err) {
                    console.error('Error accessing camera:', err);
                }
            }

            async function captureAndPredict() {
                const context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                const imageData = canvas.toDataURL('image/jpeg');
                const response = await fetch('/', {
                    method: 'POST',
                    body: JSON.stringify({ image: imageData }),
                    headers: {
                        'Content-Type': 'application/json'
                    }
                });
                const result = await response.json();
                predictionDiv.innerHTML = JSON.stringify(result, null, 2);
            }

            setInterval(captureAndPredict, 1000);

            initCamera();
        </script>
    </body>
    </html>
    """

if __name__ == '__main__':
    app.run(host='0.0.0.0')


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/May/2024 08:52:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 08:52:02] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/May/2024 08:52:02] "POST / HTTP/1.1" 200 -


24-05-31 08:52:02 - Searching [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]] in 34 length datastore


INFO:werkzeug:127.0.0.1 - - [31/May/2024 08:52:03] "POST / HTTP/1.1" 200 -


24-05-31 08:52:03 - Searching [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]] in 34 length datastore


...

24-05-31 08:52:45 - Searching [[[129 143 139]
  [129 143 139]
  [129 143 137]
  ...
  [255 255 255]
  [255 254 255]
  [255 254 255]]

 [[129 143 139]
  [129 143 139]
  [129 143 137]
  ...
  [255 255 255]
  [255 254 255]
  [255 254 255]]

 [[129 143 139]
  [129 143 139]
  [128 142 136]
  ...
  [255 255 255]
  [255 254 255]
  [255 254 255]]

 ...

 [[155 166 158]
  [155 166 158]
  [154 165 155]
  ...
  [131 145 143]
  [132 146 142]
  [132 146 142]]

 [[154 167 159]
  [153 166 158]
  [153 167 156]
  ...
  [135 149 145]
  [134 148 144]
  [133 147 143]]

 [[153 166 158]
  [152 165 157]
  [151 165 154]
  ...
  [134 148 144]
  [132 146 142]
  [130 144 140]]] in 34 length datastore


INFO:werkzeug:127.0.0.1 - - [31/May/2024 08:52:46] "POST / HTTP/1.1" 200 -


24-05-31 08:52:46 - Searching [[[129 143 137]
  [127 141 135]
  [126 140 134]
  ...
  [255 254 255]
  [253 255 254]
  [252 255 251]]

 [[128 142 136]
  [127 141 135]
  [126 140 134]
  ...
  [255 254 255]
  [253 255 254]
  [252 255 251]]

 [[127 141 135]
  [128 142 136]
  [127 141 135]
  ...
  [255 255 255]
  [253 255 254]
  [252 255 251]]

 ...

 [[183 204 195]
  [181 202 193]
  [184 203 194]
  ...
  [198 209 206]
  [196 207 204]
  [195 206 203]]

 [[184 205 196]
  [180 201 192]
  [181 203 191]
  ...
  [199 210 207]
  [197 208 205]
  [196 207 204]]

 [[187 208 199]
  [180 201 192]
  [180 202 190]
  ...
  [200 211 208]
  [198 209 206]
  [198 209 206]]] in 34 length datastore


INFO:werkzeug:127.0.0.1 - - [31/May/2024 08:52:47] "POST / HTTP/1.1" 200 -


24-05-31 08:52:47 - Searching [[[137 143 142]
  [137 143 142]
  [135 144 141]
  ...
  [254 255 253]
  [254 255 253]
  [254 255 253]]

 [[137 143 142]
  [137 143 142]
  [135 144 141]
  ...
  [254 255 253]
  [254 255 253]
  [254 255 253]]

 [[135 144 141]
  [135 144 141]
  [135 144 141]
  ...
  [255 255 254]
  [254 255 253]
  [254 255 253]]

 ...

 [[194 207 205]
  [191 204 202]
  [189 203 199]
  ...
  [202 211 208]
  [202 211 208]
  [201 210 207]]

 [[195 208 206]
  [190 203 201]
  [188 199 196]
  ...
  [202 210 209]
  [202 210 209]
  [202 210 209]]

 [[195 208 206]
  [189 202 200]
  [186 197 194]
  ...
  [202 210 209]
  [202 210 209]
  [202 210 209]]] in 34 length datastore
